In [1]:
import pandas as pd
import re
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score, classification_report, log_loss
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix


# Database connection setup
db_config = {
    'host': 'localhost',
    'database': 'postgres',
    'user': 'postgres',
    'password': 'kaviyam123',
    'port': '5432'
}
connection_string = f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}"
engine = create_engine(connection_string)

# Load data from PostgreSQL
query = 'SELECT * FROM public.overall_cleaned_base_and_pr_ef_policyef;'
df = pd.read_sql(query, con=engine)

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

df_filtered = df[df['Policy Status'].isin(['Renewed', 'Not Renewed'])]

df_filtered['Status_Binary'] = df_filtered['Policy Status'].apply(lambda x: 1 if x == 'Not Renewed' else 0)

# Select relevant columns
selected_columns = ['renewal type', 'product name', 'product name 2',  'biztype', 'age', 'vehicle idv', 'before gst add-on gwp', 'total od premium', 'total tp premium', 'gst', 
 'total premium payable', 'ncb % previous year', 'applicable discount with ncb', 'tie up',
 'Number of claims', 'approved', 'denied', 'Policy Tenure', 'Customer Tenure', 'New Customers', 'Claim Happaned/Not', 
 'Renewal Rate Status', 'withdrawn', 'policy_wise_purchase', 'Status_Binary']

df_selected = df_filtered[selected_columns]

# Convert numerical columns to float
numerical_columns = ['age', 'vehicle idv', 'before gst add-on gwp', 'total od premium', 'total tp premium', 'gst', 
 'total premium payable', 'ncb % previous year', 'applicable discount with ncb',
 'Number of claims', 'approved', 'denied', 'Policy Tenure', 'Customer Tenure', 'withdrawn', 'policy_wise_purchase', 'Status_Binary']


df_selected[numerical_columns] = df_selected[numerical_columns].apply(pd.to_numeric, errors='coerce')

# One-hot encode categorical columns
categorical_columns = ['renewal type', 'product name', 'product name 2',  'biztype', 'tie up', 'New Customers', 'Claim Happaned/Not', 
 'Renewal Rate Status']

df_selected = pd.get_dummies(df_selected, columns=categorical_columns, drop_first=True)

# Impute missing values
imputer = SimpleImputer(strategy='constant', fill_value=0)
df_selected = pd.DataFrame(imputer.fit_transform(df_selected), columns=df_selected.columns)

# Convert to sparse matrix
X = csr_matrix(df_selected.drop(columns=['Status_Binary']).values)
y = df_selected['Status_Binary']

# Debugging dimensions
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

C:\Users\Admin\AppData\Local\Temp\ipykernel_13860\3725740938.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Status_Binary'] = df_filtered['Policy Status'].apply(lambda x: 1 if x == 'Not Renewed' else 0)
C:\Users\Admin\AppData\Local\Temp\ipykernel_13860\3725740938.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected[numerical_columns] = df_selected[numerical_columns].apply(pd.to_numeric, errors='coerce')


Shape of X: (1503014, 47)
Shape of y: (1503014,)


In [ ]:
import numpy as np
import pandas as pd
import re
import json
from collections import defaultdict
from catboost import CatBoostClassifier

# Feature dependencies
all_feature_dependencies = defaultdict(list)

# Prepare feature names
feature_names = df_selected.drop(columns=['Status_Binary']).columns.tolist()

# Train CatBoost model
print("\nTraining CatBoost Model - Single Run")

cat_model = CatBoostClassifier(
    depth=10, learning_rate=0.1, iterations=500,
    random_seed=42, verbose=0
)

cat_model.fit(X, y)

# **🔹 Extract tree dump (all trees)**
model_file = "catboost_model.json"
cat_model.save_model(model_file, format="json")

# Load JSON model
with open(model_file, "r") as file:
    model_data = json.load(file)

formatted_trees = []

# **🔹 Process each tree separately**
for tree_index, tree in enumerate(model_data["oblivious_trees"]):
    formatted_tree = []
    indent = "    "  # Indentation for formatting

    print(f"Processing tree {tree_index + 1}/{len(model_data['oblivious_trees'])}...")

    for split_index, split in enumerate(tree["splits"]):
        feature_index = split["float_feature_index"]  # Feature index used in split
        threshold = split["border"]  # Threshold value for splitting
        feature_name = feature_names[feature_index]  # Map index to actual feature name

        # Format tree structure like XGBoost output
        condition = f"f{feature_index}<{threshold}"
        formatted_condition = f"{feature_name} < {threshold}"
        formatted_line = f"|--- Split {split_index}: {formatted_condition}"
        
        formatted_tree.append(indent + formatted_line)

    # **🔹 Convert leaf values into class labels**
    leaf_values = tree["leaf_values"]
    for leaf_index, value in enumerate(leaf_values):
        logit = float(value)
        probability = 1 / (1 + np.exp(-logit))  # Sigmoid function
        predicted_class = 1 if probability > 0.5 else 0
        formatted_tree.append(f"{indent}|--- Leaf {leaf_index}: class {predicted_class}")

    formatted_trees.append("\n".join(formatted_tree))

# **🔹 Save formatted trees**
output_file = "catboost_tree_run.txt"
with open(output_file, "w") as file:
    file.write("\n\n".join(formatted_trees))

print(f"Decision trees saved as '{output_file}'")


Training XGBoost Model - Single Run


AttributeError: 'CatBoostClassifier' object has no attribute 'get_booster'

In [4]:
import numpy as np
import pandas as pd
import re
import json
from collections import defaultdict
from catboost import CatBoostClassifier

# Feature dependencies
all_feature_dependencies = defaultdict(list)

# Prepare feature names
feature_names = df_selected.drop(columns=['Status_Binary']).columns.tolist()

# Train CatBoost model
print("\nTraining CatBoost Model - Single Run")

cat_model = CatBoostClassifier(
    depth=10, learning_rate=0.1, iterations=500,
    random_seed=42, verbose=0
)

cat_model.fit(X, y)

# **🔹 Extract tree dump (all trees)**
model_file = "catboost_model.json"
cat_model.save_model(model_file, format="json")

# Load JSON model
with open(model_file, "r") as file:
    model_data = json.load(file)

formatted_trees = []

# **🔹 Process each tree separately**
for tree_index, tree in enumerate(model_data["oblivious_trees"]):
    formatted_tree = []
    indent = "    "  # Indentation for formatting

    print(f"Processing tree {tree_index + 1}/{len(model_data['oblivious_trees'])}...")

    for split_index, split in enumerate(tree["splits"]):
        feature_index = split["float_feature_index"]  # Feature index used in split
        threshold = split["border"]  # Threshold value for splitting
        feature_name = feature_names[feature_index]  # Map index to actual feature name

        # Format tree structure like XGBoost output
        condition = f"f{feature_index}<{threshold}"
        formatted_condition = f"{feature_name} < {threshold}"
        formatted_line = f"|--- Split {split_index}: {formatted_condition}"
        
        formatted_tree.append(indent + formatted_line)

    # **🔹 Convert leaf values into class labels**
    leaf_values = tree["leaf_values"]
    for leaf_index, value in enumerate(leaf_values):
        logit = float(value)
        probability = 1 / (1 + np.exp(-logit))  # Sigmoid function
        predicted_class = 1 if probability > 0.5 else 0
        formatted_tree.append(f"{indent}|--- Leaf {leaf_index}: class {predicted_class}")

    formatted_trees.append("\n".join(formatted_tree))

# **🔹 Save formatted trees**
output_file = "catboost_tree_run.txt"
with open(output_file, "w") as file:
    file.write("\n\n".join(formatted_trees))

print(f"Decision trees saved as '{output_file}'")


Training CatBoost Model - Single Run
Processing tree 1/500...
Processing tree 2/500...
Processing tree 3/500...
Processing tree 4/500...
Processing tree 5/500...
Processing tree 6/500...
Processing tree 7/500...
Processing tree 8/500...
Processing tree 9/500...
Processing tree 10/500...
Processing tree 11/500...
Processing tree 12/500...
Processing tree 13/500...
Processing tree 14/500...
Processing tree 15/500...
Processing tree 16/500...
Processing tree 17/500...
Processing tree 18/500...
Processing tree 19/500...
Processing tree 20/500...
Processing tree 21/500...
Processing tree 22/500...
Processing tree 23/500...
Processing tree 24/500...
Processing tree 25/500...
Processing tree 26/500...
Processing tree 27/500...
Processing tree 28/500...
Processing tree 29/500...
Processing tree 30/500...
Processing tree 31/500...
Processing tree 32/500...
Processing tree 33/500...
Processing tree 34/500...
Processing tree 35/500...
Processing tree 36/500...
Processing tree 37/500...
Processin

In [5]:
import re
from collections import defaultdict

# Load the tree rules from the file
file_path = "catboost_tree_run.txt"

with open(file_path, "r") as file:
    tree_rules = file.readlines()

# 🔹 Extract feature names dynamically from df_selected
df_selected_columns = df_selected.columns.tolist()

# Ensure exact feature name matching
def match_feature_name(extracted_name):
    for col in df_selected_columns:
        if extracted_name.strip().lower() in col.lower():  
            return col  
    return extracted_name  

# Dictionary to store feature dependencies and their frequency
feature_threshold_counts = defaultdict(lambda: {"<": defaultdict(int), ">=": defaultdict(int)})

# Parse tree and extract rules leading to `class: 1`
def extract_non_renewal_conditions(tree_rules):
    current_path = []  # Track active decision path

    for line in tree_rules:
        line = line.strip()
        if not line:
            continue

        depth = line.count("|---")
        line_content = line.replace("|--- ", "").strip()

        # Handle decision nodes (conditions)
        match = re.search(r"([A-Za-z0-9_ /-]+) (<|>=) ([0-9\.]+)", line_content)
        if match:
            feature, operator, threshold = match.groups()
            threshold = float(threshold)

            # Ensure full feature names
            feature = match_feature_name(feature)

            current_path = current_path[:depth]  # Trim path to current depth
            current_path.append((feature, operator, threshold))  # Add new condition

        # Handle leaf nodes (class: 1)
        elif "class: 1" in line_content:
            for feature, operator, threshold in current_path:
                feature_threshold_counts[feature][operator][threshold] += 1

    return feature_threshold_counts

# Extract dependencies and their frequency
feature_threshold_counts = extract_non_renewal_conditions(tree_rules)

# Save reasons
output_file = "non_renewal_thresholds_fixed_cat.txt"

with open(output_file, "w") as file:
    file.write("Key Non-Renewal Conditions (Auto-Extracted Feature Names):\n")
    file.write("=" * 100 + "\n\n")

    for feature, threshold_data in feature_threshold_counts.items():
        most_common_less_than = max(threshold_data["<"], key=threshold_data["<"].get, default=None)
        most_common_greater_than = max(threshold_data[">="], key=threshold_data[">="].get, default=None)

        less_than_count = threshold_data["<"].get(most_common_less_than, 0)
        greater_than_count = threshold_data[">="].get(most_common_greater_than, 0)

        file.write(f" **{feature}** influences non-renewal due to:\n")

        if most_common_less_than is not None:
            file.write(f"   - {feature} < {most_common_less_than} (Count: {less_than_count})\n")

        if most_common_greater_than is not None:
            file.write(f"   - {feature} >= {most_common_greater_than} (Count: {greater_than_count})\n")

        file.write("-" * 100 + "\n")

print(f"\nUpdated non-renewal thresholds saved to '{output_file}'")


Updated non-renewal thresholds saved to 'non_renewal_thresholds_fixed_cat.txt'


In [ ]:
import re
from collections import defaultdict

# Load the tree rules from the file
file_path = "catboost_tree_run.txt"

with open(file_path, "r") as file:
    tree_rules = file.readlines()

# Automatically get feature names from dataset
df_selected_columns = [
    "Renewal Rate Status_Null", "age", "gst", "Tie Up_HYUNDAI", "Before GST Add-on GWP", "Vehicle IDV",
    "Total OD Premium", "biztype_Roll Over", "Policy Tenure", "Applicable Discount with NCB", "Tie Up_MARUTI",
    "Total TP Premium", "Approved", "Renewal Type_TY Onwards", "Renewal Type_SY onwards", "Product name _PrivateCarPolicy",
    "biztype_Renewal Business", "Tie Up_MIBL OEM", "Tie Up_Non-OEM", "Tie Up_TATA PV",
    "Product name _Standalone Own Damage Policy for Private Car", "Product name _Private Car Policy - Bundled Cover",
    "Tie Up_EM Non OE Dealership", "New Customers_Yes", "Number of claims", "Customer Tenure", "Total Premium Payable",
    "Tie Up_FORD", "Tie Up_HONDA PV", "Tie Up_SKODA", "Tie Up_VOLKSWAGEN", "Denied", "WITHDRAWN",
    "Tie Up_EM Broker", "Claim Happened/Not_Yes", "NCB % Previous Year", "OEM", "Renewal Rate Status_Increase",
    "Renewal Type_SY", "Not_Yes", "Tie Up_MIBL Others"
]  

# Ensure exact feature name matching
def match_feature_name(extracted_name):
    for col in df_selected_columns:
        if extracted_name in col:  
            return col  
    return extracted_name  

# Dictionary to store feature dependencies and their frequency
feature_threshold_counts = defaultdict(lambda: {"<": defaultdict(int), ">=": defaultdict(int)})

# Parse tree and extract rules leading to `class: 1`
def extract_non_renewal_conditions(tree_rules):
    current_path = []  

    for line in tree_rules:
        line = line.strip()
        if not line:
            continue

        depth = line.count("|---")
        line_content = line.replace("|--- ", "").strip()

        # Handle decision nodes (conditions)
        match = re.search(r"([A-Za-z0-9_ /-]+) (<|>) ([0-9\.]+) yes=(\d+),no=(\d+)", line_content)
        if match:
            feature, operator, threshold, yes_branch, no_branch = match.groups()
            threshold = float(threshold)

            # Ensure full feature names
            feature = match_feature_name(feature)

            current_path.append((depth, feature, operator, threshold, yes_branch, no_branch))

        # Handle leaf nodes (class 1)
        elif "class: 1" in line_content:
            for depth, feature, operator, threshold, yes_branch, no_branch in current_path:
                if str(yes_branch) in line:  # `<` condition
                    feature_threshold_counts[feature]["<"][threshold] += 1
                elif str(no_branch) in line:  # `>=` condition
                    feature_threshold_counts[feature][">="][threshold] += 1

    return feature_threshold_counts

# Extract dependencies and their frequency
feature_threshold_counts = extract_non_renewal_conditions(tree_rules)

# Save reasons
output_file = "non_renewal_thresholds_fixed.txt"

with open(output_file, "w") as file:
    file.write("Key Non-Renewal Conditions (Full Feature Names Fixed):\n")
    file.write("=" * 100 + "\n\n")

    for feature, threshold_data in feature_threshold_counts.items():
        most_common_less_than = max(threshold_data["<"], key=threshold_data["<"].get, default=None)
        most_common_greater_than = max(threshold_data[">="], key=threshold_data[">="].get, default=None)

        less_than_count = threshold_data["<"].get(most_common_less_than, 0)
        greater_than_count = threshold_data[">="].get(most_common_greater_than, 0)

        file.write(f" **{feature}** influences non-renewal due to:\n")

        if most_common_less_than is not None:
            dominant_marker = "" if less_than_count > greater_than_count else ""
            file.write(f"   {dominant_marker}- {feature} < {most_common_less_than} (Count: {less_than_count})\n")

        if most_common_greater_than is not None:
            dominant_marker = "" if greater_than_count > less_than_count else ""
            file.write(f"   {dominant_marker}- {feature} >= {most_common_greater_than} (Count: {greater_than_count})\n")

        file.write("-" * 100 + "\n")

print(f"\nUpdated non-renewal thresholds saved to '{output_file}'")

In [7]:
import re
from collections import defaultdict

# Load the tree rules from the file
file_path = "catboost_tree_run.txt"

with open(file_path, "r") as file:
    tree_rules = file.readlines()

# Extract feature names dynamically from df_selected
df_selected_columns = df_selected.columns.tolist()

# 🔹 Ensure exact feature name matching
def match_feature_name(extracted_name):
    for col in df_selected_columns:
        if extracted_name.strip().lower() in col.lower():  
            return col  
    return extracted_name  # Return original if no match found

# 🔹 Dictionary to store feature dependencies and their frequency
feature_threshold_counts = defaultdict(lambda: {"<": defaultdict(int), ">=": defaultdict(int)})

# 🔹 Parse tree and extract rules leading to `class: 1`
def extract_non_renewal_conditions(tree_rules):
    current_path = []  # Track active decision path
    leaf_class_1_found = False  # Flag to track when a path leads to `class: 1`

    for line in tree_rules:
        line = line.strip()
        if not line:
            continue

        depth = line.count("|---")
        line_content = line.replace("|--- ", "").strip()

        # 🔹 Handle decision nodes (conditions)
        match = re.search(r"([A-Za-z0-9_ /-]+) (<|>=) ([0-9\.]+)", line_content)
        if match:
            feature, operator, threshold = match.groups()
            threshold = float(threshold)

            # Ensure full feature names
            feature = match_feature_name(feature)

            # Trim path if going back up in tree depth
            current_path = current_path[:depth]  
            current_path.append((feature, operator, threshold))  # Add new condition

        # 🔹 Handle leaf nodes (class: 1)
        elif "class: 1" in line_content:
            leaf_class_1_found = True  # Mark that we found a valid class: 1 leaf

            for feature, operator, threshold in current_path:
                feature_threshold_counts[feature][operator][threshold] += 1

    return feature_threshold_counts, leaf_class_1_found

# 🔹 Extract dependencies and their frequency
feature_threshold_counts, leaf_class_1_found = extract_non_renewal_conditions(tree_rules)

# 🔹 Save results
output_file = "non_renewal_thresholds_fixed.txt"

with open(output_file, "w") as file:
    file.write("Key Non-Renewal Conditions (Auto-Extracted Feature Names):\n")
    file.write("=" * 100 + "\n\n")

    if not leaf_class_1_found:
        file.write("No significant non-renewal conditions found. Check if `class: 1` exists in the tree.\n")
    else:
        for feature, threshold_data in feature_threshold_counts.items():
            most_common_less_than = max(threshold_data["<"], key=threshold_data["<"].get, default=None)
            most_common_greater_than = max(threshold_data[">="], key=threshold_data[">="].get, default=None)

            less_than_count = threshold_data["<"].get(most_common_less_than, 0)
            greater_than_count = threshold_data[">="].get(most_common_greater_than, 0)

            file.write(f" **{feature}** influences non-renewal due to:\n")

            if most_common_less_than is not None:
                file.write(f"   - {feature} < {most_common_less_than} (Count: {less_than_count})\n")

            if most_common_greater_than is not None:
                file.write(f"   - {feature} >= {most_common_greater_than} (Count: {greater_than_count})\n")

            file.write("-" * 100 + "\n")

print(f"\nUpdated non-renewal thresholds saved to '{output_file}'")



Updated non-renewal thresholds saved to 'non_renewal_thresholds_fixed.txt'


In [8]:
import re
from collections import defaultdict

# 🔹 Load the tree rules from the file
file_path = "catboost_tree_run.txt"

with open(file_path, "r") as file:
    tree_rules = file.readlines()

# 🔹 Extract feature names dynamically from df_selected
df_selected_columns = df_selected.columns.tolist()

# 🔹 Ensure exact feature name matching
def match_feature_name(extracted_name):
    for col in df_selected_columns:
        if extracted_name.strip().lower() in col.lower():  
            return col  
    return extracted_name  # Return original if no match found

# 🔹 Dictionary to store feature dependencies and their frequency
feature_threshold_counts = defaultdict(lambda: {"<": defaultdict(int), ">=": defaultdict(int)})

# 🔹 Parse tree and extract rules leading to `class: 1` ("Not Renewed")
def extract_non_renewal_conditions(tree_rules):
    current_path = []  # Stack to track active decision path

    for line in tree_rules:
        line = line.strip()
        if not line:
            continue

        depth = line.count("|---")  # Determine depth in the tree
        line_content = line.replace("|--- ", "").strip()

        # 🔹 Handle decision nodes (conditions)
        match = re.search(r"([A-Za-z0-9_ /-]+) (<|>=) ([0-9\.]+)", line_content)
        if match:
            feature, operator, threshold = match.groups()
            threshold = float(threshold)

            # Ensure full feature names
            feature = match_feature_name(feature)

            # Trim path if going back up in tree depth
            current_path = current_path[:depth]  
            current_path.append((feature, operator, threshold))  # Add new condition

        # 🔹 Handle leaf nodes (`class: 1` -> Not Renewed)
        elif "class: 1" in line_content:
            # Record the full decision path leading to `class: 1`
            for feature, operator, threshold in current_path:
                feature_threshold_counts[feature][operator][threshold] += 1

    return feature_threshold_counts

# 🔹 Extract dependencies and their frequency
feature_threshold_counts = extract_non_renewal_conditions(tree_rules)

# 🔹 Save results
output_file = "non_renewal_thresholds_fixed.txt"

with open(output_file, "w") as file:
    file.write("Key Non-Renewal Conditions (Auto-Extracted Feature Names):\n")
    file.write("=" * 100 + "\n\n")

    if not feature_threshold_counts:
        file.write("No significant non-renewal conditions found. Check if `class: 1` exists in the tree.\n")
    else:
        for feature, threshold_data in feature_threshold_counts.items():
            most_common_less_than = max(threshold_data["<"], key=threshold_data["<"].get, default=None)
            most_common_greater_than = max(threshold_data[">="], key=threshold_data[">="].get, default=None)

            less_than_count = threshold_data["<"].get(most_common_less_than, 0)
            greater_than_count = threshold_data[">="].get(most_common_greater_than, 0)

            file.write(f" **{feature}** contributes to non-renewal due to:\n")

            if most_common_less_than is not None:
                file.write(f"   - {feature} < {most_common_less_than} (Count: {less_than_count})\n")

            if most_common_greater_than is not None:
                file.write(f"   - {feature} >= {most_common_greater_than} (Count: {greater_than_count})\n")

            file.write("-" * 100 + "\n")

print(f"\nUpdated non-renewal thresholds saved to '{output_file}'")


Updated non-renewal thresholds saved to 'non_renewal_thresholds_fixed.txt'


In [9]:
import re
from collections import defaultdict

# 🔹 Load the tree rules from the file
file_path = "catboost_tree_run.txt"

with open(file_path, "r") as file:
    tree_rules = file.readlines()

# 🔹 Extract feature names dynamically from df_selected
df_selected_columns = df_selected.columns.tolist()

# 🔹 Ensure exact feature name matching
def match_feature_name(extracted_name):
    extracted_name = extracted_name.strip().lower()

    for col in df_selected_columns:
        if extracted_name in col.lower():  
            return col  # Return full dataset feature name

    print(f"⚠ Warning: Feature name `{extracted_name}` not found in dataset!")
    return extracted_name  # Return original if no match found

# 🔹 Dictionary to store feature dependencies and their frequency
feature_threshold_counts = defaultdict(lambda: {"<": defaultdict(int), ">=": defaultdict(int)})

# 🔹 Parse tree and extract rules leading to `class: 1` ("Not Renewed")
def extract_non_renewal_conditions(tree_rules):
    current_path = []  # Stack to track active decision path
    class_1_found = False  # Flag to check if we detect class: 1

    for line in tree_rules:
        line = line.strip()
        if not line:
            continue

        depth = line.count("|---")  # Determine depth in the tree
        line_content = line.replace("|--- ", "").strip()

        # 🔹 Handle decision nodes (conditions)
        match = re.search(r"([A-Za-z0-9_ /-]+) (<|>=) ([0-9\.]+)", line_content)
        if match:
            feature, operator, threshold = match.groups()
            threshold = float(threshold)

            # Ensure full feature names
            feature = match_feature_name(feature)

            # Trim path if going back up in tree depth
            current_path = current_path[:depth]  
            current_path.append((feature, operator, threshold))  # Add new condition

        # 🔹 Handle leaf nodes (`class: 1` -> Not Renewed)
        elif "class: 1" in line_content:
            class_1_found = True
            print(f"🚀 Found `class: 1` at depth {depth}, path: {current_path}")

            # Record the full decision path leading to `class: 1`
            for feature, operator, threshold in current_path:
                feature_threshold_counts[feature][operator][threshold] += 1

    return feature_threshold_counts, class_1_found

# 🔹 Extract dependencies and their frequency
feature_threshold_counts, class_1_found = extract_non_renewal_conditions(tree_rules)

# 🔹 Save results
output_file = "non_renewal_thresholds_fixed.txt"

with open(output_file, "w") as file:
    file.write("Key Non-Renewal Conditions (Auto-Extracted Feature Names):\n")
    file.write("=" * 100 + "\n\n")

    if not class_1_found:
        file.write("No significant non-renewal conditions found. Check if `class: 1` exists in the tree.\n")
    else:
        for feature, threshold_data in feature_threshold_counts.items():
            most_common_less_than = max(threshold_data["<"], key=threshold_data["<"].get, default=None)
            most_common_greater_than = max(threshold_data[">="], key=threshold_data[">="].get, default=None)

            less_than_count = threshold_data["<"].get(most_common_less_than, 0)
            greater_than_count = threshold_data[">="].get(most_common_greater_than, 0)

            file.write(f" **{feature}** contributes to non-renewal due to:\n")

            if most_common_less_than is not None:
                file.write(f"   - {feature} < {most_common_less_than} (Count: {less_than_count})\n")

            if most_common_greater_than is not None:
                file.write(f"   - {feature} >= {most_common_greater_than} (Count: {greater_than_count})\n")

            file.write("-" * 100 + "\n")

print(f"\nUpdated non-renewal thresholds saved to '{output_file}'")



Updated non-renewal thresholds saved to 'non_renewal_thresholds_fixed.txt'


In [10]:
import re
from collections import defaultdict

# Train your CatBoostClassifier
cat_model.fit(X, y)

# Extract the tree dumps directly (each tree is represented as a string)
tree_dumps = cat_model.get_tree_dumps()

# Optionally, write the tree dumps to a file so you can process them later
dump_file = "catboost_tree_run.txt"
with open(dump_file, "w") as f:
    for tree in tree_dumps:
        f.write(tree + "\n")  # separate trees by newlines

# Load the tree rules from the file
with open(dump_file, "r") as file:
    tree_rules = file.readlines()

# 🔹 Extract feature names dynamically from df_selected
df_selected_columns = df_selected.columns.tolist()

# Ensure exact feature name matching
def match_feature_name(extracted_name):
    for col in df_selected_columns:
        if extracted_name.strip().lower() in col.lower():
            return col
    return extracted_name

# Dictionary to store feature dependencies and their frequency
feature_threshold_counts = defaultdict(lambda: {"<": defaultdict(int), ">=": defaultdict(int)})

# Parse tree dump and extract rules leading to `class: 1`
def extract_non_renewal_conditions(tree_rules):
    current_path = []  # Track active decision path
    for line in tree_rules:
        line = line.strip()
        if not line:
            continue

        depth = line.count("|---")
        line_content = line.replace("|--- ", "").strip()

        # Handle decision nodes (conditions)
        match = re.search(r"([A-Za-z0-9_ /-]+) (<|>=) ([0-9\.]+)", line_content)
        if match:
            feature, operator, threshold = match.groups()
            threshold = float(threshold)

            # Ensure full feature names
            feature = match_feature_name(feature)

            current_path = current_path[:depth]  # Trim path to current depth
            current_path.append((feature, operator, threshold))  # Add new condition

        # Handle leaf nodes (e.g., indicating class: 1)
        elif "class: 1" in line_content:
            for feature, operator, threshold in current_path:
                feature_threshold_counts[feature][operator][threshold] += 1

    return feature_threshold_counts

# Extract dependencies and their frequency from the tree dump
feature_threshold_counts = extract_non_renewal_conditions(tree_rules)

# Save the extracted rules to a file
output_file = "non_renewal_thresholds_fixed_cat.txt"
with open(output_file, "w") as file:
    file.write("Key Non-Renewal Conditions (Auto-Extracted Feature Names):\n")
    file.write("=" * 100 + "\n\n")

    for feature, threshold_data in feature_threshold_counts.items():
        most_common_less_than = max(threshold_data["<"], key=threshold_data["<"].get, default=None)
        most_common_greater_than = max(threshold_data[">="], key=threshold_data[">="].get, default=None)

        less_than_count = threshold_data["<"].get(most_common_less_than, 0)
        greater_than_count = threshold_data[">="].get(most_common_greater_than, 0)

        file.write(f" **{feature}** influences non-renewal due to:\n")

        if most_common_less_than is not None:
            file.write(f"   - {feature} < {most_common_less_than} (Count: {less_than_count})\n")

        if most_common_greater_than is not None:
            file.write(f"   - {feature} >= {most_common_greater_than} (Count: {greater_than_count})\n")

        file.write("-" * 100 + "\n")

print(f"\nUpdated non-renewal thresholds saved to '{output_file}'")

AttributeError: 'CatBoostClassifier' object has no attribute 'get_tree_dumps'

In [11]:
import catboost
print(catboost.__version__)

1.2.7


In [12]:
import numpy as np
import pandas as pd
import json
from collections import defaultdict
from catboost import CatBoostClassifier

# Prepare feature names from your DataFrame (excluding target 'Status_Binary')
feature_names = df_selected.drop(columns=['Status_Binary']).columns.tolist()

# Train CatBoost model
print("\nTraining CatBoost Model - Single Run")
cat_model = CatBoostClassifier(
    depth=10, learning_rate=0.1, iterations=500,
    random_seed=42, verbose=0
)
cat_model.fit(X, y)

# Save the model in JSON format
model_file = "catboost_model.json"
cat_model.save_model(model_file, format="json")

# Load the JSON model
with open(model_file, "r") as file:
    model_data = json.load(file)

# Dictionaries to store the decision paths ("reasons") for each class
reasons_for_class = {0: [], 1: []}

# Process each tree from the JSON model
for tree_index, tree in enumerate(model_data["oblivious_trees"]):
    splits = tree["splits"]       # List of splits for this tree
    num_splits = len(splits)      # d splits, so 2^d leaves
    num_leaves = len(tree["leaf_values"])
    
    # For each leaf in the tree, reconstruct the decision path
    for leaf_index in range(num_leaves):
        # Create a binary representation of the leaf index, padded to the number of splits
        path_bin = format(leaf_index, f"0{num_splits}b")
        conditions = []
        
        # Build conditions based on each bit in the path
        for j, bit in enumerate(path_bin):
            feature_index = splits[j]["float_feature_index"]
            threshold = splits[j]["border"]
            feature_name = feature_names[feature_index]
            if bit == '0':
                # Left branch: condition satisfied if feature < threshold
                conditions.append(f"{feature_name} < {threshold}")
            else:
                # Right branch: condition satisfied if feature >= threshold
                conditions.append(f"{feature_name} >= {threshold}")
                
        # Compute predicted probability for this leaf using the sigmoid of the logit
        logit = float(tree["leaf_values"][leaf_index])
        probability = 1 / (1 + np.exp(-logit))
        predicted_class = 1 if probability > 0.5 else 0
        
        # Combine the conditions into one readable decision path
        decision_path = " AND ".join(conditions)
        reasons_for_class[predicted_class].append(decision_path)

# Save the decision paths (reasons) to a file
output_file = "decision_paths_by_class.txt"
with open(output_file, "w") as file:
    file.write("Reasons (Decision Paths) leading to Class 1:\n")
    file.write("=" * 50 + "\n")
    for reason in reasons_for_class[1]:
        file.write(reason + "\n")
        
    file.write("\n\nReasons (Decision Paths) leading to Class 0:\n")
    file.write("=" * 50 + "\n")
    for reason in reasons_for_class[0]:
        file.write(reason + "\n")

print(f"\nDecision paths saved to '{output_file}'")



Training CatBoost Model - Single Run

Decision paths saved to 'decision_paths_by_class.txt'


In [13]:
import numpy as np
import json
from collections import defaultdict
from catboost import CatBoostClassifier

# Prepare feature names from your DataFrame (excluding target 'Status_Binary')
feature_names = df_selected.drop(columns=['Status_Binary']).columns.tolist()

# Train CatBoost model
cat_model = CatBoostClassifier(
    depth=10, learning_rate=0.1, iterations=500,
    random_seed=42, verbose=0
)
cat_model.fit(X, y)

# Save the model in JSON format
model_file = "catboost_model.json"
cat_model.save_model(model_file, format="json")

# Load the JSON model
with open(model_file, "r") as file:
    model_data = json.load(file)

# Create dictionaries to count condition occurrences for each class
condition_counts = {0: defaultdict(int), 1: defaultdict(int)}

# Process each tree from the JSON model
for tree in model_data["oblivious_trees"]:
    splits = tree["splits"]       # List of splits for this tree
    num_splits = len(splits)      # d splits, so 2^d leaves
    num_leaves = len(tree["leaf_values"])
    
    # For each leaf, reconstruct its decision path and count conditions
    for leaf_index in range(num_leaves):
        path_bin = format(leaf_index, f"0{num_splits}b")
        conditions = []
        for j, bit in enumerate(path_bin):
            feature_index = splits[j]["float_feature_index"]
            threshold = splits[j]["border"]
            feature_name = feature_names[feature_index]
            if bit == '0':
                condition = f"{feature_name} < {threshold}"
            else:
                condition = f"{feature_name} >= {threshold}"
            conditions.append(condition)
        
        # Determine the predicted class for this leaf using sigmoid
        logit = float(tree["leaf_values"][leaf_index])
        probability = 1 / (1 + np.exp(-logit))
        predicted_class = 1 if probability > 0.5 else 0
        
        # Increase count for each unique condition in the path
        # (you can also choose to count combinations if needed)
        for cond in set(conditions):
            condition_counts[predicted_class][cond] += 1

# Now, filter and output the main conditions (for example, top 10 for each class)
top_n = 10
main_reasons = {}
for cls in (0, 1):
    sorted_conditions = sorted(condition_counts[cls].items(), key=lambda x: x[1], reverse=True)
    main_reasons[cls] = sorted_conditions[:top_n]

# Write the main reasons to a file
output_file = "main_decision_reasons.txt"
with open(output_file, "w") as file:
    for cls in (0, 1):
        file.write(f"Main Conditions for Class {cls}:\n")
        file.write("=" * 40 + "\n")
        for condition, count in main_reasons[cls]:
            file.write(f"{condition}  (Count: {count})\n")
        file.write("\n")

print(f"Main decision reasons saved to '{output_file}'")


Main decision reasons saved to 'main_decision_reasons.txt'
